<a href="https://colab.research.google.com/github/parkerburchett/EconCapstone/blob/main/Notebook_to_Gather_Ethermine_Transaction_data_and_do_Cleaning_and_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is the first draft of the notebook I am writing for my Economics Capstone to get a published paper. \

https://www.etherchain.org/charts Look at "mining revenue" gives USD value per day of a GH/s. Use this instead of building it yourself.
### Outline: What will this notebook do?

1. Use the Etherscan.io charts to convert into production 
* Block Number to Date.(Year, Month)
* (Year, Month) to Monthly average Value of 1 day of mining at 1 GH/s (Etherscan.io chart). Just average by month. 

* (Year, Month) to Average ETH Price. 


* You will need a dictionary that looks like 
(Year, Month): Average Gh/s needed to mine make 1 ETH 
### Called production dictionary. 

2. You will need to write custom Estimate_hashrate 

*Pseudocode:
  Estimate_HashRate(Year, Month, ETH earned this month):

  estimated_hashrate_in_ghS = Production_Dictionary[(Year, Month) * ETH earned  this month)


3. Use the Etherscan.io API to get all of the transactions from 5 large pools, and store that in pyspark on Google Colab. You will need to save that to someplace in the cloud so that you can get it later and put the public link. I think drop box is the right place for it.

 -> What I want the data to look like.
 This information is factual raw data. 
(Block Number of Transaction, Pool Address, Miner Address, Amount of ETH)


Every Single Transaction will be mapped into this form. 

(Block Number, Day, Year, Month, Pool Address, Miner Address, Amount of ETH, Estimated_hashrate)

This data will then be grouped by Miner_address and Month.

Intermediate(after group by) result:

Month, Year, Pool Address, Miner Address, ETH Earned this month, Estimated Hashrate, (ETH earned this month * Average Monthly Price of ETH) as monthly USD Revenue.

Frame work for Miner Features
This is how I am choosing to categorize Miners. 

## Miner Features
Start month ( cast this as an int with January 2015 as 1). # month of first  income).
End Month (cast this as an int with March 2021 as the 80 or whatever the number happens to be). Last month with income. or + infinity if that was march 2021. Age as number of full months,
Median Hashrate, as size. 



Dummy Varibles for Pool address, 
Dummy Variables for if they went to a different




Framework for OLS regression:

##Independent variable: 
* Estimated Monthly Hashrate.

###Dependent Variables
Miner Features. 
Year
Month,
Average ETH price,
average Gh/s value



### Setup Pyspark and Connect it to your google drive

In [94]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Open Connection to Google drive

from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("\n\n DRIVE SETUP")

openjdk-8-jdk-headless is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 DRIVE SETUP


## Install Libraries 

In [95]:
import pandas as pd
import numpy as np
import json
import requests
import math
import datetime
import matplotlib.pyplot as plt
import time
%matplotlib inline

import pyspark
from functools import reduce
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import DataType
from pyspark import SparkContext, SparkConf


### Create Spark Session

In [96]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050") # copy and pasted this, dont' know what it means
# create the context
# try catch block lets you start again from the top without causing problems. 
try:
  sc.stop()
  sc = pyspark.SparkContext(conf=conf)
except:
  sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

print('Successfully Created Spark Session')

Successfully Created Spark Session


## Methods to get transactions from Etherscan.io API


### You will want some try catch logic to make sure that you are not needlessly pinging Etherscan.io

In [167]:
def read_api_key():
  api_key_json = open(r'/content/drive/MyDrive/etherScan_apiKey.json') # read in my private Etherscan.io API key from my Google Drive
  etherscan_api_key =json.load(api_key_json)['key']
  api_key_json.close()
  return etherscan_api_key

ETHERSCAN_API_KEY = read_api_key() # You api key is a constant so the variable name is UPPER CASE

def query_normal_transactions(wallet_address, startblock=0, endblock=99999999): # returning a list here might be inefficient. 
  """
    Ping Etherscan.io and get the most recent 10k transactions for this wallet.
    Each of these takes ~4 seconds
  """
  api_request_text =f'https://api.etherscan.io/api?module=account&action=txlist&address={wallet_address}&startblock={startblock}&endblock={endblock}&sort=asc&apikey={ETHERSCAN_API_KEY}'
     
  response = requests.get(api_request_text) # Need to throttle this to 5 calls a second
  api_response_text = json.loads(response.text) 
  # error handling to make sure that the call is going through
  if api_response_text['message'] != 'OK': # used
    print(f'you got an error at address={wallet_address} \nstartblock= {startblock}\nendblock={endblock}')

  else:
    transaction_list = api_response_text['result']
    simplified_transactions = parse_normal_transactions(transaction_list=transaction_list, wallet_address=wallet_address)

    #exclude the last N transactions since the might overlapp
    largest_block = simplified_transactions[-1][2] # O(1)
    while simplified_transactions[-1][2] == largest_block:
      simplified_transactions.pop(-1) # O(1) * 100 upper bound.

    column_names = ['to_address',
                    'from_address',
                    'block_number',
                    'transaction_date',
                    'value_in_ether']
    
    # create a rdd of this batch of transactions
    rdd = spark.createDataFrame(simplified_transactions, schema=column_names)
    return rdd, largest_block

def parse_normal_transactions(transaction_list: list, wallet_address):
    """
      Converts transaction data into a easy to read from and excludes the last block
    :param transaction_list: A list of dictionary objects. Each dictionary is a single transaction.
    :return:
        simple_transactions: a list of tuples storing (to_address, from_address, Block number, Value in Ether)
                                      Data types are: (String, String, int, DateTime, float)
    """
    simplified_transactions =[(trans_dict['to'],
                              trans_dict['from'],
                              int(trans_dict['blockNumber']),
                              datetime.datetime.fromtimestamp(int(trans_dict['timeStamp'])),
                              int(trans_dict['value'])/math.pow(10, 18))
                              for trans_dict in transaction_list if trans_dict['from'] == wallet_address
                              ]
    return simplified_transactions

### Trying to make the code marginally faster. 




In [168]:
%%time
ethermine_transaction_rdd, largest_block = query_normal_transactions(wallet_address=ethermine_wallet)
for i in range(9):
  time.sleep(.2) # makes it exter certain that I will not overwhelm the API call limit. adds approx 20 min to the total time cost. Is worth
  next_chunk_rdd, largest_block = query_normal_transactions(wallet_address=ethermine_wallet, startblock=largest_block)
  ethermine_transaction_rdd = ethermine_transaction_rdd.union(next_chunk_rdd)
  print(f'call {i+1}') 

print('You made 100 calls and you ought to ~100,000 Records. Should take 4 minutes')
print(ethermine_transaction_rdd.count())


call 1
call 2
call 3
call 4
call 5
call 6
call 7
call 8
call 9
You made 100 calls and you ought to ~100,000 Records. Should take 4 minutes
99981
CPU times: user 6.54 s, sys: 384 ms, total: 6.92 s
Wall time: 41.9 s


[Row(to_address='0x42bf2074cb179309290284392ca5445c0cd0a0af', from_address='0xea674fdde714fd979de3edf0f56aa9716b898ec8', block_number=1536846, transaction_date=datetime.datetime(2016, 5, 18, 0, 37, 58), value_in_ether=1.002161848778827),
 Row(to_address='0x43c0a04a58290d7e2da48ab2e08c46f38f23a87c', from_address='0xea674fdde714fd979de3edf0f56aa9716b898ec8', block_number=1536849, transaction_date=datetime.datetime(2016, 5, 18, 0, 38, 23), value_in_ether=0.09812615348406149),
 Row(to_address='0xc5f0addc119947597be8fa295d10432b2e5cbbe3', from_address='0xea674fdde714fd979de3edf0f56aa9716b898ec8', block_number=1536850, transaction_date=datetime.datetime(2016, 5, 18, 0, 38, 51), value_in_ether=1.0000600065502974),
 Row(to_address='0x8ac65d4f3192d234f5bb6813fcde997f9c7adfb5', from_address='0xea674fdde714fd979de3edf0f56aa9716b898ec8', block_number=1536850, transaction_date=datetime.datetime(2016, 5, 18, 0, 38, 51), value_in_ether=1.0716547809514696),
 Row(to_address='0xf6e8d6a872f84fb6131f23bc8

## Known Wallets.

In [92]:
# Constants
parker_wallet = '0x76fb6d38f28c44a13380220df21363bd7af45ee1'
ethan_wallet = '0xceb4d0ca821420cf2553b9e244f6b52364613f94'
ethermine_wallet = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'


TypeError: ignored

### Right now it takes 5 seconds per call. You need to speed that up a massive amount


Start at start_block = 0 and end block =999999999

### next

In [160]:
# How many API calls do you need to make?
n_transactions = 40000000
reccords_per_call = 10000
total_api_calls = n_transactions/reccords_per_call
base_time_cost_per_api_call = 5 #Loose Uppperbound
seconds_needed = (base_time_cost_per_api_call *total_api_calls) 
hours = seconds_needed/3600
print(f'Upper bound for API calls for ethermine {total_api_calls_needed}')
print(f'Upper bound for hours to get all of Ethermine hours to get everything from ethermine {hours}')

Upper bound for API calls for ethermine 4000.0
Upper bound for hours to get all of Ethermine hours to get everything from ethermine 5.555555555555555
